In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import KFold

# 데이터셋 디렉토리 설정
data_dir = '/Users/songhune/Downloads/data/kth'
actions = ['walking']
# 'boxing', 'handclapping', 'handwaving', 'jogging', 'running',
# 비디오 파일을 읽어 numpy 배열로 변환
def read_videos(action):
    video_files = [os.path.join(data_dir, action, f) for f in os.listdir(os.path.join(data_dir, action)) if f.endswith('.avi')]
    clips = []
    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 그레이스케일로 변환
            frames.append(frame)
        cap.release()
        clips.append(np.array(frames))
    return clips

# 데이터셋 준비
clips = []
labels = []
for idx, action in enumerate(actions):
    action_clips = read_videos(action)
    clips.extend(action_clips)
    labels.extend([idx] * len(action_clips))

# 모든 클립의 길이를 동일하게 맞추기 위해 패딩
max_frames = max(clip.shape[0] for clip in clips)
padded_clips = []
for clip in clips:
    pad_width = ((0, max_frames - clip.shape[0]), (0, 0), (0, 0))
    padded_clips.append(np.pad(clip, pad_width, mode='constant'))

clips = np.array(padded_clips)
labels = np.array(labels)

# K-Fold를 사용하여 데이터셋 분할
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_indices, test_indices = next(kf.split(clips))

train_clips = clips[train_indices]
train_labels = labels[train_indices]
test_clips = clips[test_indices]
test_labels = labels[test_indices]

# clips 배열 생성 함수
def create_clips_array(clips, clip_length):
    clips_array = []
    for i in range(len(clips)):
        for j in range(0, clips[i].shape[0], clip_length):
            if j + clip_length <= clips[i].shape[0]:
                clips_array.append([j, clip_length])
    return np.array(clips_array)

# dims 배열 생성 (1, 120, 160) - KTH 데이터셋 스타일
dims = np.array([1, clips.shape[2], clips.shape[3]])

# input_raw_data 배열 생성 함수
def create_input_raw_data(clips, clip_length):
    raw_data = []
    for clip in clips:
        for i in range(0, clip.shape[0], clip_length):
            if i + clip_length <= clip.shape[0]:
                raw_data.extend(clip[i:i+clip_length])
    return np.array(raw_data).reshape(-1, 1, clips.shape[2], clips.shape[3])

clip_length = 10  # Moving MNIST와 유사하게 각 클립의 길이를 10 프레임으로 설정

# Training 데이터셋 저장
train_clips_array = create_clips_array(train_clips, clip_length)
train_input_raw_data = create_input_raw_data(train_clips, clip_length)
np.savez('/Users/songhune/Downloads/data/kth/train.npz', clips=train_clips_array, dims=dims, input_raw_data=train_input_raw_data)

# Test 데이터셋 저장
test_clips_array = create_clips_array(test_clips, clip_length)
test_input_raw_data = create_input_raw_data(test_clips, clip_length)
np.savez('/Users/songhune/Downloads/data/kth/test.npz', clips=test_clips_array, dims=dims, input_raw_data=test_input_raw_data)

print('Data has been successfully split and saved to training.npz and test.npz.')


Data has been successfully split and saved to training.npz and test.npz.
